In [1]:
import pandas as pd
import glob 
import numpy as np

### 5.1

Read the in-sample data from the end of 2000 to the end of 2007 into Python (8 estimation windows), modify the variable names to a single word, create a new column/variable ‘year’ in each dataset for the sample period, and aggregate all the data files into one DataFrame. Remove the missing observations

### In-Sample Construction

Import Excel Files (In-Sample)

In [2]:
files = glob.glob('lab5*.xlsx') #glob to get an easy list of all files in a folder
files.sort() # comes out random so we sort 
files

['lab5_data_2000_2001.xlsx',
 'lab5_data_2001_2002.xlsx',
 'lab5_data_2002_2003.xlsx',
 'lab5_data_2003_2004.xlsx',
 'lab5_data_2004_2005.xlsx',
 'lab5_data_2005_2006.xlsx',
 'lab5_data_2006_2007.xlsx',
 'lab5_data_2007_2008.xlsx',
 'lab5_data_2008_2009.xlsx',
 'lab5_data_2009_2010.xlsx']

In [3]:
files_is = files[:8] # construct in-sample files
files_oos = files[8:] # construct out-of-sample files

In [4]:
li = []
for filename in files_is:
    raw = pd.read_excel(filename, index_col=None, header=0)
    raw.columns = ["name", "ticker", "exchange", "clsgPrice", "size", "beMe", "eP", "m12M", "yr1Ret", "ind"]
    raw["year"] = int(filename[10:14]) #consider changing the numbers '10:14' to be in accordance with the file name
    
    li.append(raw)

df = pd.concat(li, axis=0, ignore_index=True)

C:\Users\pvile\miniconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\pvile\miniconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\pvile\miniconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\pvile\miniconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\pvile\miniconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook con

In [5]:
df.head()

,name,ticker,exchange,clsgPrice,size,beMe,eP,m12M,yr1Ret,ind,year
0,"1-800 Contacts, Inc.",CTAC,NASDAQ,28.62500,331195457.0,0.042162,0.022707,1.057143,-0.565415,43.0,2000
1,"1-800 Flowers.com, Inc.",FLWS,NASDAQ,4.12500,264772047.0,0.474933,-0.276364,-0.761905,2.781818,43.0,2000
2,"1st Century Bancshrs, Inc.",FCTY,NASDAQ,NaN,NaN,NaN,NaN,NaN,NaN,45.0,2000
3,1st Constitution Bancorp,FCCY,NASDAQ,3.14115,NaN,NaN,NaN,-0.380140,1.009090,45.0,2000
4,"1st Independence Financial Group, Inc.",FIFG,NASDAQ,14.18800,21493046.0,1.152143,0.053566,-0.001561,-0.184455,45.0,2000


In [6]:
df.set_index(["name", "ticker", "exchange", "year"], inplace = True) #apply an index removes the values from further calculations

Drop missing values

In [7]:
df.dropna(inplace = True) #check to make sure all missing values need to go

In [8]:
df.shape

(37844, 7)

### 5.2

Create a routine that for each estimation window performs the following:

(a) Remove any outliers by creating an index that keeps only the rows in each variable that contain data
values within 3 standard deviations about the mean.

(b) Filter the data to keep only those stocks with positive book values, positive earnings, a closing price greater than 5, and a with market capitalization greater than $100MM.

(c) Create a new variable for the log of market value of equity called lnSize

(d) Expresses the in-sample factor exposures (log of the market value of equity, book-to-market equity, earnings-to-price, and 1-year price momentum) in terms of z-scores.

(e) Run multiple multivariate, cross-sectional regressions to calculate the Ordinary Least Squares (OLS) estimate of the factor return premia for each year using the stock’s factor exposure z-scores. Save the estimates of the factor return premia of each estimation year into a Dataframe called coefficients.


c) Create log size (a few steps early)

In [9]:
df["lnSize"] = df['size'].apply(np.log)

a. Get rid of outliers

In [10]:
#removing outliers
def is_out(x):
    return abs(x - x.mean()) > x.std() * 3

#creating subset of our variables
out = df.loc[:,["lnSize", "beMe", "eP", "m12M", "yr1Ret"]]
out.index = out.index.get_level_values('year')
out.reset_index(inplace = True)

#applying out is_out function
outA = out.groupby('year').apply(is_out)
outA.drop('year',axis =1, inplace = True)
outA.apply(sum, axis = 0)

#creating boolean index
outB = outA.any(axis = 1)

#using boolean index
df = df.reset_index()[~outB]

print(df.shape)

(37046, 12)


b. Remove negative beMes, eP, stocks under 5 and size under 100,000,000

In [11]:
df = df[(df['beMe'] > 0) & (df['eP'] > 0) & (df['clsgPrice'] > 5) & (df['size'] > 1e8)] 

d. Create Z-Scores

In [12]:
df.shape

(21116, 12)

In [13]:
factors = ['beMe', 'eP', 'm12M', 'lnSize']

In [14]:
zfactors = []
for i in factors:
    df['z'+ i] = df.groupby(['year', 'ind'])[i].transform(lambda x: (x - x.mean()) / x.std())
    zfactors.append('z'+i)

e. Multivariate Cross-sectional Regression

In [15]:
from sklearn.linear_model import LinearRegression #decided to use sklearn's linear model as opposed to statmodels

In [16]:
df_group = df.groupby('year') #create a grouped data frame to group by year

Creating a DatFrame of intercpets and coefficients

I had to use a fairly round about way. I may check again to see if there is an easier way but this is the best I could come up with thus far.

In [17]:
li = []
for group in df_group.groups.keys(): # for each group key
    df = df_group.get_group(group) # construct a separate dataframe
    X = df[zfactors] # using z factors for our X
    y = df['yr1Ret'] # returns for Y
    model = LinearRegression() # instantiate our model
    results = model.fit(X,y) # fit and we will construct our dataframe of results
    
    # below I am taking the empty list from above and putting the coefficient
    li.append(pd.Series([group, results.intercept_, results.coef_[0], results.coef_[1], results.coef_[2], results.coef_[3]]))
    
    coefficients = pd.concat(li, axis=1, ignore_index=True) # concat all the lists of Series
    
"""
ALTERNATIVE WAY:
coefficients = pd.DataFrame()
for i in df['year']:
    results = smf.ols(formula = 'yr1Ret ~ zlnSize + zbeMe + zeP + zm12M', data = df[df['year'] == i]).fit()
    coeff = pd.DataFrame(results.params)
    coeff.columns = [i]
    coefficients = pd.concat([coefficients, coeff], axis = 1)
coefficients = coefficients.T
"""

# clean up: I transpose, put labels on the data and change the year to an integer (e.g. it comes out 2001.0)
coefficients = coefficients.T
coefficients.columns = ['year', 'intercept'] + zfactors
coefficients['year'] = coefficients['year'].astype(int)
coefficients

,year,intercept,zbeMe,zeP,zm12M,zlnSize
0,2000,0.102504,0.034567,0.037881,-0.030944,-0.091730
1,2001,-0.087476,0.024294,0.008882,0.049394,-0.024826
2,2002,0.413593,0.059811,0.033395,0.015052,-0.020659
3,2003,0.211030,0.023203,0.033438,0.006857,-0.007462
4,2004,0.098329,0.021566,-0.001643,0.022159,0.014301
5,2005,0.186823,0.027151,0.008255,-0.008139,-0.004530
6,2006,0.033146,-0.024208,-0.011649,0.017497,0.032552
7,2007,-0.354561,-0.017075,-0.010986,-0.030622,-0.011614


### 5.3

Aggregate the periodic backtests into overall diagnostic statistics using the Fama-MacBeth (FM, 1973) procedure.

(a) Compute the estimated factor return premia vector, which is the simple element-by-element time-series average of the coefficients across all estimation windows. Show the results.

(b) Calculate the standard deviation vector of the coefficients and show the results.

(c) Calculate the FM t-statistic for any given factor premia estimate. The FM t-statistic is the average of the coefficient estimate divided by its standard deviation, multiplied by the square root of the number of estimation windows. Show the results.

(d) Explain (1) what the estimated factor return premia vector and (2) the FM t-statistics represent.

In [18]:
coefficients.set_index("year",inplace = True)
coef_mean = coefficients.mean()
coef_std  = coefficients.std()
coef_tstat = coef_mean / coef_std * np.sqrt(8)

print('t_stat')
print('------')
print(coef_tstat)

t_stat
------
intercept    0.941187
zbeMe        1.937953
zeP          1.699443
zm12M        0.532102
zlnSize     -1.104932
dtype: float64


The estimated factor return premia vector represents the average premium rewarded for the exposure to a particular risk factor. 

In addition, the FM t-statistic represents how significant or robust is the factor return comparing the factor return premia vector to its standard deviation among the periods (and the square root of years). A significant factor return premium vector is one with a higher average number but also with a lower deviation. 

We may compute the FM t-statistics when we want to check if a factor premium is relevant across many years individually. If a factor premium doesn't change across many years, we can conclude that the factor premium is pervasive. For example, we don't want to get a factor premium that appears significantly only for a few years. 

### 5.4
	
Select the factors for the stock return model that will be tested over the forecasting windows, based on whether (a) the value of the FM t-statistics is statistically significant (i.e., t-statistic  ≥ 1.5) and (b) whether the signs of the FM t-statistics for the factors are in accordance with what we expect from theory. In the case that none of the factors meet these criteria, for the purposes of completing the following problems, select the factor that has the correct sign and the most significant FM t-statistic. 


__We will select beMe and eP.__

### 5.5

Redo part 5.1 to 5.3 for the out-of-sample test. Create a rountine that for each foreasting window performs the following:

(a) Import and clean for missing and erroneous entries for the out-of-sample data.

(b) Remove any outliers by creating an index that keeps only the rows in each variable that contain data values within 3 standard deviations about the mean.

(c) Filter the data to keep only those stocks with positive book values, positive earnings, a closing price greater than \\$5, and a market capitalization greater than $100MM.

(d) Create a new variable for the log of the market value of equity called lnSize.

(e) Express the out-of-sample factor exposures (log of the market value of equity, book-to-market equity, earnings-to-price, and 1-year price momentum) in terms of z-scores.

(f) Based on the stock return model **with the selected factors from 5.4**, create the overall FM t-statistic weighted z-scores for these factors, called **scores**, which will be used to score each of the
stocks in the forecasting window. The overall score is calculated by weighting each stock’s
factor exposure z-score for the period ending at date t by the FM t-statistic for that factor derived
in problem 5.3 and then adding those weighted z-scores together for a stock: z_factor1 *
FM_tstat_factor1 + z_factor2 * FM_tstat_factor2 and so on.

(g) To construct our long-short portfolio, we need to **rank** the scores based on the **score** column from the previous step. The **rank** should list the companies in deciles for **each year** based on the **scores**. (Hint: Check out the functions/methods *transform/apply* and *qcut*).

(h) List the individual stocks' performance of the long-short portfolio.

### Out-of-Sample Construction

### a. 
Import and clean for missing or erroneous entries for the out-of-sample data. 

In [19]:
li = []
for filename in files_oos:
    raw = pd.read_excel(filename, index_col=None, header=0)
    raw.columns = ["name", "ticker", "exchange", "clsgPrice", "size", "beMe", "eP", "m12M", "yr1Ret", "ind"]
    raw["year"] = int(filename[10:14])
    
    li.append(raw)

df_oos = pd.concat(li, axis=0, ignore_index=True)

C:\Users\pvile\miniconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\pvile\miniconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [20]:
#df_oos.set_index(["name", "ticker", "exchange"], inplace = True)

In [21]:
df_oos.dropna(inplace = True) #check to make sure all missing values need to go

### d.
Create log size variable. (two steps early)

In [22]:
df_oos["lnSize"] = df_oos['size'].apply(np.log)

### b. 
Remove any outliers by creating an index that keeps only the rows in each variable that contain datavalues within 3 standard deviations about the mean.

In [23]:
#removing outliers
def is_out(x):
    return abs(x - x.mean()) > x.std() * 3

#creating subset of our variables
out = df_oos.loc[:,["beMe", "eP"]] 
out.head()

#applying out is_out function
outA = out.apply(is_out)
outA.apply(sum, axis = 0)

#creating boolean index
outB = outA.any(axis = 1)
outB

#using boolean index
df_oos = df_oos[~outB]

print(df_oos.shape)

(9424, 12)


### c. 
Filter the data to keep only those stocks with positive book values, positive earnings, a closing price greater than \\$5, and a market capitalization greater than $100MM.

In [24]:
df_oos = df_oos[(df_oos['beMe'] > 0) & (df_oos['eP'] > 0) & (df_oos['clsgPrice'] > 5) & (df_oos['size'] > 1e8)] 

### e. 
Express the out-of-sample factor exposures (log of the market value of equity, book-to-market equity, earnings-to-price, and 1-year price momentum) in terms of z-scores.

In [25]:
df_oos

,name,ticker,exchange,clsgPrice,size,beMe,eP,m12M,yr1Ret,ind,year,lnSize
7,1st Source Corporation,SRCE,NASDAQ,23.63,5.697413e+08,0.796263,0.049937,0.134351,-0.294094,45.0,2008,20.160693
17,3M Co,MMM,"NEW YORK STOCK EXCHANGE, INC.",57.54,3.987263e+10,0.247789,0.091936,-0.173980,0.485925,12.0,2008,24.408956
18,"3PAR, Inc.",PAR,"NEW YORK STOCK EXCHANGE, INC.",7.63,4.637376e+08,0.263539,0.009174,-0.435028,0.553080,35.0,2008,19.954830
19,"3SBio, Inc. ADR",SSRX,NASDAQ,7.80,1.677846e+08,0.803987,0.051344,-0.618866,0.755128,13.0,2008,18.938192
20,"51job, Inc. ADR",JOBS,NASDAQ,6.16,1.736447e+08,1.035503,0.070602,-0.656250,1.886347,34.0,2008,18.972522
...,...,...,...,...,...,...,...,...,...,...,...,...
17719,iRobot Corporation,IRBT,NASDAQ,17.60,4.405709e+08,0.302324,0.008523,0.350575,0.413636,34.0,2009,19.903582
17731,"infoGROUP, Inc. B",IUSA,NASDAQ,8.02,4.628732e+08,0.541908,0.011222,1.061069,-0.002494,34.0,2009,19.952964
17733,j2 Global Inc,JCOM,NASDAQ,20.35,9.000220e+08,0.373514,0.075676,0.018442,0.422604,32.0,2009,20.617930
17736,"rue21, Inc.",RUE,NASDAQ,28.09,6.804792e+08,0.048334,0.029548,0.000000,0.043432,43.0,2009,20.338308


In [26]:
factors = ["beMe", "eP"] #could include the other factors since they won't be considered in the ran
zfactors = []
for factor in factors: 
    from sklearn.preprocessing import scale
    df_oos['z'+ factor] = df_oos.groupby(['year', 'ind'])[factor].apply(lambda x: (x - x.mean()) / x.std())
    print('z' + factor + " Mean: " + str(round(df['z' + factor].mean(),2)))
    print('z' + factor + " STD: " + str(round(df['z' + factor].std(),2)))
    
    zfactors.append('z' + factor)

zbeMe Mean: 0.0
zbeMe STD: 0.99
zeP Mean: -0.0
zeP STD: 0.99


### f. 
Based on the stock return model **with the selected factors from 5.4**, create the overall FM t-statistic weighted z-scores for these factors, called **scores**, which will be used to score each of the stocks in the forecasting window. The overall score is calculated by weighting each stock’s factor exposure z-score for the period ending at date t by the FM t-statistic for that factor derived in problem 5.3 and then adding those weighted z-scores together for a stock: z_factor1 * FM_tstat_factor1 + z_factor2 * FM_tstat_factor2 and so on.

In [27]:
df_oos['score'] = df_oos['zeP'] * coef_tstat['zeP'] + df_oos['zbeMe'] * coef_tstat['zbeMe']

### g. 
To construct our long-short portfolio, we need to **rank** the scores based on the **score** column from the previous step. The **rank** should list the companies in deciles for **each year** based on the **scores**. (Hint: Check out the functions/methods transform/apply and qcut).

In [28]:
df_oos['rank'] = df_oos.groupby('year')['score'].apply(lambda x: pd.qcut(x, 10, labels = False))
df_oos.head(2)

,name,ticker,exchange,clsgPrice,size,beMe,eP,m12M,yr1Ret,ind,year,lnSize,zbeMe,zeP,score,rank
7,1st Source Corporation,SRCE,NASDAQ,23.63,5.697413e+08,0.796263,0.049937,0.134351,-0.294094,45.0,2008,20.160693,-0.172605,-0.264452,-0.783921,4
17,3M Co,MMM,"NEW YORK STOCK EXCHANGE, INC.",57.54,3.987263e+10,0.247789,0.091936,-0.173980,0.485925,12.0,2008,24.408956,-0.849253,-0.040268,-1.714245,2


### h.

List the individual stocks performance of the long-short portfolio.

In [29]:
long = df_oos[df_oos['rank'] == 9]
short = df_oos[df_oos['rank'] == 0]

short.reset_index(inplace = True)
long.reset_index(inplace = True)

### 5.6
### a.
Upload the ret.csv file. It has the ticker information and the monthly returns for 2009 and 2010

In [30]:
ret = pd.read_csv('ret.csv')

ret.columns = ['permno', 'date', 'ticker', 'ret']

### b.

Transform the ‘date’ column to the **datetime** format and create new ‘year’ and ‘month’ columns. 

In [31]:
ret['date'] = pd.to_datetime(ret['date'], format = '%Y%m%d')

In [32]:
ret.date.unique()

array(['2009-01-30T00:00:00.000000000', '2009-02-27T00:00:00.000000000',
       '2009-03-31T00:00:00.000000000', '2009-04-30T00:00:00.000000000',
       '2009-05-29T00:00:00.000000000', '2009-06-30T00:00:00.000000000',
       '2009-07-31T00:00:00.000000000', '2009-08-31T00:00:00.000000000',
       '2009-09-30T00:00:00.000000000', '2009-10-30T00:00:00.000000000',
       '2009-11-30T00:00:00.000000000', '2009-12-31T00:00:00.000000000',
       '2010-01-29T00:00:00.000000000', '2010-02-26T00:00:00.000000000',
       '2010-03-31T00:00:00.000000000', '2010-04-30T00:00:00.000000000',
       '2010-05-28T00:00:00.000000000', '2010-06-30T00:00:00.000000000',
       '2010-07-30T00:00:00.000000000', '2010-08-31T00:00:00.000000000',
       '2010-09-30T00:00:00.000000000', '2010-10-29T00:00:00.000000000',
       '2010-11-30T00:00:00.000000000', '2010-12-31T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [33]:
ret['year'] = ret.date.apply(lambda x: x.year)
ret['month'] = ret.date.apply(lambda x: x.month)

### c.
Compute the long and short returns separately and then calculate the long-short overall return, which is the difference between the long portfolio return and the short portfolio return.

In [34]:
li = []
for i in range(2008,2010):
    longRet = ret[(ret.ticker.isin(long[long.year==i].ticker))&(ret.year==i+1)]
    longRet = longRet.groupby(['year', 'month']).ret.mean()
    longRet = longRet.reset_index()
    longRet.columns = ['year', 'month', 'long']
    
    shortRet = ret[(ret.ticker.isin(short[short.year==i].ticker))&(ret.year==i+1)]
    shortRet = shortRet.groupby(['year', 'month']).ret.mean()
    shortRet = shortRet.reset_index()
    shortRet.columns = ['year', 'month', 'short']
    
    output = pd.merge(longRet, shortRet, left_on = ['year', 'month'], right_on = ['year', 'month'])
    
    li.append(output)

port = pd.concat(li, axis=0, ignore_index=True)
port['LS'] = port['long'] - port['short']

In [35]:
port.head()

,year,month,long,short,LS
0,2009,1,-0.063086,-0.080979,0.017893
1,2009,2,-0.149375,-0.098047,-0.051328
2,2009,3,0.127424,0.081342,0.046083
3,2009,4,0.330767,0.124439,0.206328
4,2009,5,0.132963,0.031934,0.101030


### d. 
Upload the FF.csv and transform the ‘date’ column to the datetime format and create new ‘year’ and ‘month’ columns. Merge the long-short return dataframe with the FF by the ‘year’ and ‘month’ columns. 

In [36]:
ff = pd.read_csv('FF.csv')

ff['date'] = ff.dateff.apply(lambda x: pd.to_datetime(x, format='%Y%m%d'))
ff['year'] = ff.date.apply(lambda x: x.year)
ff['month'] = ff.date.apply(lambda x: x.month)

ff.head()

,dateff,mktrf,smb,hml,rf,umd,date,year,month
0,20050131,-0.0276,-0.0151,0.0197,0.0016,0.0321,2005-01-31,2005,1
1,20050228,0.0189,-0.0051,0.0163,0.0016,0.0316,2005-02-28,2005,2
2,20050331,-0.0197,-0.0139,0.0158,0.0021,0.0054,2005-03-31,2005,3
3,20050429,-0.0261,-0.0399,-0.0034,0.0021,-0.0085,2005-04-29,2005,4
4,20050531,0.0365,0.0289,-0.0081,0.0024,0.0043,2005-05-31,2005,5


In [37]:
port = pd.merge(port, ff, left_on = ['year', 'month'], right_on = ['year', 'month'])

In [38]:
port

,year,month,long,short,LS,dateff,mktrf,smb,hml,rf,umd,date
0,2009,1,-0.063086,-0.080979,0.017893,20090130,-0.0812,-0.0001,-0.1110,0.0000,-0.0183,2009-01-30
1,2009,2,-0.149375,-0.098047,-0.051328,20090227,-0.1010,0.0016,-0.0725,0.0001,0.0418,2009-02-27
2,2009,3,0.127424,0.081342,0.046083,20090331,0.0895,-0.0008,0.0352,0.0002,-0.1138,2009-03-31
3,2009,4,0.330767,0.124439,0.206328,20090430,0.1019,0.0483,0.0546,0.0001,-0.3439,2009-04-30
4,2009,5,0.132963,0.031934,0.101030,20090529,0.0521,-0.0233,-0.0021,0.0000,-0.1244,2009-05-29
5,2009,6,0.012944,0.030889,-0.017945,20090630,0.0043,0.0261,-0.0271,0.0001,0.0529,2009-06-30
6,2009,7,0.137001,0.066854,0.070148,20090731,0.0772,0.0207,0.0527,0.0001,-0.0536,2009-07-31
7,2009,8,0.049560,0.011722,0.037838,20090831,0.0333,-0.0090,0.0776,0.0001,-0.0884,2009-08-31
8,2009,9,0.088159,0.063667,0.024493,20090930,0.0408,0.0245,0.0092,0.0001,-0.0493,2009-09-30
9,2009,10,-0.059916,-0.034743,-0.025173,20091030,-0.0259,-0.0423,-0.0418,0.0000,0.0265,2009-10-30


### e.
e)	Calculate the simple mean, geometric mean, and annualized Sharpe Ratio of the (i) long return, (ii) short return, and (iii) long-short returns. 

In [41]:
print('Simple Mean')
print(port[['long', 'short', 'LS']].mean())
print('===================================')

import statistics
print('Geometric Mean')
print(port[['long', 'short', 'LS']].apply(lambda x: x+1).apply(statistics.geometric_mean).apply(lambda x: x-1))
print('===================================')

print('Sharpe Ratio')
sr = port.LS.mean() / port.LS.std()
print(sr)
print('===================================')

print('Annualized Sharpe Ratio')
print(sr * np.sqrt(12))
print('===================================')

Simple Mean
long     0.040575
short    0.022648
LS       0.017927
dtype: float64
Geometric Mean
long     0.036257
short    0.020827
LS       0.016704
dtype: float64
Sharpe Ratio
0.34092149713903475
Annualized Sharpe Ratio
1.1809867088745116


### f.
Now that we have our long-short portfolio, we can evaluate our portfolio against the (i) CAPM, (ii) Fama-French Three-Factor Model, and the (iii) Carhart Four-Factor Model by running a regression of the long-short portfolio returns with each of these three models

In [42]:
print('CAPM Factor')
import statsmodels.formula.api as sm
CAPM = sm.ols(formula = 'LS ~ mktrf', data = port).fit()
print(CAPM.summary())
print('===================================')

print('FF3 Factor')
import statsmodels.formula.api as sm
FF3 = sm.ols(formula = 'LS ~ mktrf + smb + hml', data = port).fit()
print(FF3.summary())
print('===================================')

print('C4 Factor')
C4 = sm.ols(formula = 'LS ~ mktrf + smb + hml + umd', data = port).fit()
print(C4.summary())
print('===================================')

CAPM Factor
                            OLS Regression Results                            
Dep. Variable:                     LS   R-squared:                       0.292
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     9.089
Date:                Thu, 09 Mar 2023   Prob (F-statistic):            0.00637
Time:                        11:11:12   Log-Likelihood:                 41.294
No. Observations:                  24   AIC:                            -78.59
Df Residuals:                      22   BIC:                            -76.23
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0089      0.010      0.

### 5.7
When testing investment strategies, idiosyncratic events or using a short time frame that only captures certain business cycles can disguise the true profitability of the model under study. To see this, do a second backtest for a long-short portfolio strategy that uses a fundamental factor approach, but that consists of 5 estimation periods (starting with factor exposures at end of 2003 to forecast the calendar year 2004 stock returns and ending with factor exposures at end of 2007 to forecast calendar year 2008 stock returns) and 2 forecasting periods (starting with factor exposures at end of 2008 to forecast calendar year 2009 stock returns, and ending with factor exposures at end of 2009 to forecast calendar year 2010 stock returns). Modify the code and calculate problems 5.1 through 5.6 to run the new backtest. Show the results. 

Repeat the answer of 5.1 to 5.6 with a different in-sample window. You'll probably find that none of the factors have a FM t-statistic higher than 1.5, so you should select the most significant factor with a correct sign. 

### 5.8
How do the results of the backtest performed over 5 estimation windows differ from the results of the one conducted over 8 estimation windows?  

A few points you could compare the two different backtests:
- Compare the factors selected (FM t-stat) and the return statistics (mean returns, Sharpe Ratio, etc) for each
- Compare the results you got from the evaluation of your LS portfolio against the CAPM, FF3, and C4 Factor. 